# Funksjoner

I flere av disse konsept-oppgavene er en del av oppgaven å dele tidsperioden inn i seksjoner som en måte å filtrere rådataene på. Hva om vi hadde en funksjon som gjorde dette for oss automatisk? Dette bruker vi som utgangspunkt til et eksempel og en oppgave som dreier seg om hvordan enkle funksjoner kan bygges i python. 

Å lage funksjoner er veldig praktisk når man har en operasjon man har bruk for ofte. Det er lurt å lage funsjonene sine generelle og ikke for omfattende - hvis de blir for spesifikke vil de bare fungere i helt spesielle tilfeller, og man får bruk for dem sjelden. 

In [1]:
import numpy as np # for regneoperasjoner
import matplotlib.pyplot as plt # for figurer
import matplotlib

Denne funksjonen skal
- lese inn et datasett fra github mappen
- dele datasettet inn i bolker med lengde angitt av dT (antall år)
- kunne håndtere både datasettene med års- og måneds-gjennomsnitt
- putte "rest år" i en siste bolk detsom lengden ikke går opp med dT, og skrive ut en beskjed om hvilke år som er i denne bolken (dersom aktuelt)
- jeg vil gjerne at den skal kunne sjekke om den trenger å ta vekk header osv selv, men per nå må man ha input på det.

In [6]:
def array_mean(
    file,dT,filetype, 
    filepath='https://raw.githubusercontent.com/irendundas/EkteData/main/'
):
    
    '''
    Denne funksjonen skal
    - lese inn et datasett fra github mappen til Ekte Data.
    - dele datasettet inn i bolker med lengde angitt av dT (antall år).
    - kunne håndtere både datasettene med års- og måneds-gjennomsnitt.
    - putte "rest år" i en siste bolk dersom lengden ikke går opp med dT, 
      og skrive ut en beskjed om hvilke år som er i denne bolken (dersom 
      aktuelt).
    - jeg vil gjerne at den skal kunne sjekke om den trenger å ta vekk header 
      osv selv, men per nå må man ha input på det.
    
    Input 
    file     : the name of the txt-file, not including the path, which is 
    constant and set below. 
    dT       : the timestep, i.e., the number of years to average across. 
    filetype : 'M' if the dataset has monthly resolution, or 'Y' if it has 
    yearly resolution.
    
    Output
    meanVal  : gjennomsnittsverdier i bolker med lengde bestemt av dT.
    time     : en tidsvektor med midtpunktene av hver tidsbolk.
    
    '''
    
    
    if filetype == 'Y':
        data = np.genfromtxt(filepath+file, 
                             dtype=float, delimiter=',',skip_header=1)
    elif filetype == 'M':
        data = np.genfromtxt(filepath+file, dtype=float)
        
    data[data==-999.99]=np.nan
    T=len(data) # Antall år i datasettet
    
    start=0 # startindeks
    stop=T  # stopindeks
    
    # Start- og slutt-indeks til hver av de "N" "dT"-års-bolkene
    ind=np.arange(start,stop,dT) 

    # In this case dT doesn't add up into T, and some years will be left out. We 
    # want to take the mean over these as well, and include them at the end of 
    # the averaged time series. 
    if T-ind[-1] > 0: 
        rest=T-ind[-1] # nr of years left out
    
        if rest < dT/2:
            ind[-1]=T
            print(
                'Dividing the datset into sections of length',dT,'leaves out',
                rest,'year(s) at the end of the time series. The last element in', 
                'the output is the average of these/this',rest,'year(s) combined', 
                'with the last full period.'
            )
            print((
                f'Dividing æpajspgiaj {dT} leaves out'
                f'ny kine {rest}'
            ))
        elif rest > dT/2:
            ind=np.append(ind,len(data)) 
            print(
                'Dividing the datset into sections of length',dT,'leaves out',
                rest,'year(s) at the end of the time series. The last element ',
                'in the output is the average of these/this',rest,'year(s).'
            )
    
    
    
    meanVal=np.zeros((len(ind)-1,data.shape[1]-1)) # dT-bolker x 1 eller 12 avhengig av om det skal bli års- eller månedssnitt
    
    if data.shape[1]==2:
        for i in range(len(ind)-1): 
             # for e.g., i=0 blir dette data[ind[0]:ind[1],1]=data[0:dT,1]
            meanVal[i]=np.nanmean(data[int(ind[i]):int(ind[i+1]),1]) 
            
    elif data.shape[1]==13:
        for mo in range(12): # iterer gjennom månedene
            for i in range(len(ind)-1): # iterer gjennom dT-årsperiodene
                # vi begynner på "data" sin kolonne mo+1 fordi første kolonne i "data" er årstallene 
                meanVal[i,mo]=np.nanmean(data[int(ind[i]):int(ind[i+1]),mo+1]) 
                
                
    # I tilegg til å returnere gjennomsnittsverdiene vil vi ha en vektor med tid. 
    # Dette gir midtpunktene i hver bolk. 
    timeid = np.round(np.arange(start+dT/2,stop-dT/2,dT))
    timeid = timeid.astype(int)
    time=data[timeid,0]
    
    return meanVal, time

In [7]:
array_mean(file='TempBergenYearly.txt',dT=10,filetype='Y')

Dividing the datset into sections of length 10 leaves out 1 year(s) at the end of the time series. The last element in the output is the average of these/this 1 year(s) combined with the last full period.
Dividing æpajspgiaj 10 leaves outny kine 1


(array([[7.17037037],
        [7.08333333],
        [7.28666667],
        [7.25648148],
        [7.1475    ],
        [7.25277778],
        [7.08095238],
        [7.83055556],
        [7.98541667],
        [7.72037037],
        [7.38916667],
        [7.62166667],
        [7.7462963 ],
        [8.08083333],
        [8.296875  ],
        [8.84166667]]),
 array([1866., 1876., 1886., 1896., 1906., 1916., 1926., 1936., 1946.,
        1956., 1966., 1976., 1986., 1996., 2006., 2016.]))